# Examples
**Note**: The examples are found in the following jupyter notebook `xmen/examples/examples.ipynb` where they can be run interactively. Before running the notebook make sure that `xmen` is on your `PYTHONPATH` following the steps given in Setup.

## Defining an experiment
New experiments are defined by specifying **parameters** (as public class attributes) and **execution** (by overloading the `run` method). For example

In [1]:
from xmen.experiment import Experiment
import os

class AnExperiment(Experiment):
    """The doc string of AnExperiment class"""
    
    def __init__(self):
        """The doc string of __init__ experiment class."""
        super(AnExperiment, self).__init__()
        self.a: str = 'h'     # A parameter
        self.b: int = 17      # Another parameter
            
        # Private attributes will not be treated as parameters
        self._c = None
    
    def run(self):
        print('\nThe experiment state inside run is')
        print(self.status)
        
        with open(os.path.join(self.root, self.name, 'logs.txt'), 'w') as f:
            f.write('This was written from a running experiment')

exp = AnExperiment()

As the experiment class has a metaclass `TypedMeta` all derived classes will also inheret the meta class `TypedMeta`. Each parameter should be defined alongside there type, default value and documentation as:
```python
self.a: str = 'h'     # A parameter```

This allows parameters to be defined quickly, with their type, default value and documentation inferred by consulting the `__init__` definition. Each attribute comment is automatically added to the Doc string of both the code and the class. Parameters are changed from their default values after instantiatian using the `update` method.

Experiments may assume that they have access to a repository `{root}/{name}`. They are able to log, backup or save anything they want here during the execution of the experiment.

In [2]:
exp.update({'a': 'g'})
print(exp)

status: default
created: 07:47PM September 18, 2019
messages:
parameters:
  a: g   # A parameter (default='h')
  b: 17   # Another parameter (default=17)


Here you can see that the parameter `a` has been updated to `g`. Its default value is still preserved in the docstring. You can also see the `status` of the experiment is currently `default`. 

An experiments status is used to define what an experiment is able to do. With default status an experiment status an experiment can be initialised and its parameters updated **but not** run:

In [3]:
try:
    exp()
except ValueError as e:
    print('ValueError:', e)

ValueError: An experiment in default status must be registered before it can be executed


Default experiments are however allowed to be written out to a `defaults.yml` file. This can be used to initialise a new experiment from a set of parameters different to those defined in the init method:

In [4]:
exp.to_defaults('/tmp')

In [5]:
%%bash
echo "$(cat /tmp/defaults.yml)"

created: 07:47PM September 18, 2019  #  The date the last time the parameters of the model were updated.
version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/<ipython-input-1-4ff1dba6a4a0>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
    branch: master
a: g #  A parameter (default='h')
b: 17 #  Another parameter (default=17)


Additionally, the date the `defaults.yml` was created is stored in the `created` field. The documentation for each parameter is automatically loaded from the doc string of the class. We are now able to load a new experiment from this paticular parameter setting through the `from_yml` method call:

In [6]:
# Now lets create a new experiment and load its state from the
exp2 = AnExperiment()
exp2.from_yml('/tmp/defaults.yml')
print(exp2)

status: default
created: 07:47PM September 18, 2019
messages:
version:
  module: /Users/robweston/xmen/examples/<ipython-input-1-4ff1dba6a4a0>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
    branch: master
parameters:
  a: g   # A parameter (default='h')
  b: 17   # Another parameter (default=17)


## Running an experiment
In order to be run an experiment object must first be registered to an experiment folder. This is achieved through the `register` method call. At this point the parameters of the experiment can no longer be changed. They are backed up to a `defaults.yml` inside the experiment folder. For example:

In [7]:
%%bash 
rm -f '/tmp/a/params.yml'

In [8]:
exp.register(root='/tmp', name='a', purpose='An example')

Looking inside the project repository we now have:

In [9]:
%%bash
echo 'Folder now contains:'
ls /tmp/a

Folder now contains:
logs.txt
params.yml


The `params.yml` file contains the following:

In [10]:
!echo "$(cat /tmp/a/params.yml)$"

root: /tmp  #  The root directory to which the experiment belongs (should not be set)
name: a #  The name of the experiment (should not be set)
status: registered #  The status of the experiment (one of ['default' | 'created' | 'running' | 'error' | 'finished'])
created: 07:47PM September 18, 2019 #  The date the last time the parameters of the model were updated.
purpose: An example #  The purpose for the experiment (should not be set)
messages: {} #  A dictionary of messages which are able to vary throughout the experiment (should not be set)
version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/<ipython-input-1-4ff1dba6a4a0>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
    branch: master
a: g #  A parameter (default='h')
b: 17 #  Another parameter (default=17)$


An experiment **cannot** be registered to a repository already containing a `params.yml` file:

In [11]:
exp2 = AnExperiment()
try:
    exp2.register(root='/tmp', name='a')
except ValueError as e:
    print('ValueError: ', e)

ValueError:  Experiment folder /tmp/a already contains a params.yml file. An Exeperiment cannot be created in an already existing experiment folder


However, instead an experiment object can be loaded from already existing `params.yml` using the method call `from_yaml`:

In [12]:
print(exp2, '\n')
exp2.from_yml('/tmp/a/params.yml')
print(exp2)

status: default
created: 07:47PM September 18, 2019
messages:
parameters:
  a: h   # A parameter (default='h')
  b: 17   # Another parameter (default=17) 

root: /tmp
name: a
status: registered
created: 07:47PM September 18, 2019
purpose: An example
messages:
version:
  module: /Users/robweston/xmen/examples/<ipython-input-1-4ff1dba6a4a0>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
    branch: master
parameters:
  a: g   # A parameter (default='h')
  b: 17   # Another parameter (default=17)


Now the parameter `a` has been updated to `g` (from its default `h`). When loading from an already created `params.yml` file the status of the experiment is updated to `registered`. The `created` attribute stores the time that the experiment was registered and root and name have also been filled.

> At the moment (as above) two experiments can share the same `params.yml` file. This may however be undesirable. In the future it may also be worth adding a locked attribute to the params.yml file, which is True when a params.yml file is linked to an Experiment object and False when it has been generated by the `ExperimentManager`

After being registered an experiment may now be run by calling an experiment `__call__` method. Upon entering the `__call__` method an experiments status (and `params.yml`) file is updated to `running` before calling the experiments `run()` method. On leaving `run()` the experiment status is updated to either `finished` or `error`. 

In [13]:
print('Before running:')
print(exp.status)

# Run the experiment using the __call__ method
exp()

print('\nAfter running:')
print(exp.status)

Before running:
registered

The experiment state inside run is
running

After running:
finished


In [14]:
%%bash
echo 'The logs.txt file has been written during the call'
ls '/tmp/a'
echo ''
echo "$(cat /tmp/a/logs.txt)"

The logs.txt file has been written during the call
logs.txt
params.yml

This was written from a running experiment


## Git Compatibility

As experiments are run it becomes things need to change. Parameters defaults are altered, new parameters are added, and old parameters appreciated. Config files soon become outdated and trying to work out which config file belong to which version of an experiment becomes tedious at best and impossible at worst. To help with this the experiment class has Git support built into its definition. When creating `defaults.yml` files from an experiment object, if the experiment was defined in a git repository, then the local git repo and commit corresponding to the experiment is logged is updated in the `git` attibute and saved in the `defaults.yml` file (alongiside the remote repository if available). Additionally the path to the module in which the experiment was defined and the experiment name are stored in the `definition` attribute.

A similar process occurs when an experiment is registered and saved in the `params.yml` file. Incompatibility between `defaults.yml` files used to initialise experiments and the current version of the experiment can therefore easily be checked. Likewise, it is possible to perfectly recreate an experiment run by checking out the commit defined in a `params.yml` file. When git is available this functionality is baked into the experiment class allowing the user to continue about there buisness without worrying about experiment compatibility.

If we once again look in the `params.yml` file and `defaults.yml` we created earlier we can see that as `AnExperiment` was defined in this notebook which iniself lives inside a git repo that the correct fields have been populated:


In [15]:
%%bash
echo "The params.yml file:"
echo "----------------------"
echo "$(cat /tmp/a/params.yml)"
echo ""
echo "The defaults.yml file:"
echo "----------------------"
echo "$(cat /tmp/defaults.yml)"

The params.yml file:
----------------------
root: /tmp  #  The root directory to which the experiment belongs (should not be set)
name: a #  The name of the experiment (should not be set)
status: finished #  The status of the experiment (one of ['default' | 'created' | 'running' | 'error' | 'finished'])
created: 07:47PM September 18, 2019 #  The date the last time the parameters of the model were updated.
purpose: An example #  The purpose for the experiment (should not be set)
messages: {} #  A dictionary of messages which are able to vary throughout the experiment (should not be set)
version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/<ipython-input-1-4ff1dba6a4a0>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
    branch: master
a: g #  A parameter (default='h')
b: 17 #  Another p

Alternatively these attributes can be acessed in python as:

In [16]:
from ruamel.yaml.comments import CommentedMap

exp3 = AnExperiment()
exp3.from_yml('/tmp/defaults.yml')
version_dict = exp3.version

def recursive_lines(dic):
    lines = []
    for k, v in dic.items():
        if type(v) is CommentedMap:
            lines += ['  ' + l for l in recursive_lines(v)]
        else:
            lines += [f'{k}: {v}']
    return lines

print('\n'.join(recursive_lines(version_dict)))

module: /Users/robweston/xmen/examples/<ipython-input-1-4ff1dba6a4a0>
class: AnExperiment
  local: /Users/robweston/xmen
  remote: https://github.com/robw4/xmen.git
  commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
  branch: master


## Command Line Interface
As well as designed to be run within python, each experiment is also able to expose a command line interface with minimal overhead. In order to expose the command line interface some minor additions need to be made to the definition of `AnExperiment`:

```python
# Import the experiment_parser
from xmen.experiment import Experiment, experiment_parser

class AnExperiment(Experiment):
    # ... Define an experiment

# Add the following lines to the bottom of the module
if __name__ == '__main__':
    args = experiment_parser.parse_args()
    exp = AnExperiment()
    exp.main(args)
```

The `AnExperiment` defined earlier is defined in `experiment.py` module with the above modifications. This allows an `AnExperiment` to be used to generate `default.yml`, to be registered and to be executed all from the command line:

In [17]:
%%bash
python experiment.py -h

usage: experiment.py [-h] [--update YAML_STRING] [--execute PARAMS]
                     [--to_root DIR] [--to_defaults DIR]
                     [--register ROOT NAME]

Run the Experiment command line interface

optional arguments:
  -h, --help            show this help message and exit
  --update YAML_STRING  Update the parameters given by a yaml string. Note
                        this will be called beforeother flags and can be used
                        in combination with --to_root, --to_defaults,and
                        --register.
  --execute PARAMS      Execute the experiment from the given params.yml file.
                        Cannot be called with update.
  --to_root DIR         Generate a run script and defaults.yml file for
                        interfacing with the experiment manager. If the
                        directory does not exist then it is first created.
  --to_defaults DIR     Generate a defaults.yml file from the experiment
                        

For example:

In [18]:
%%bash

# Generate a defaults.yml
python experiment.py --update '{a: updated}' --to_defaults /tmp/command_line/updated_defaults

# Visualise results
echo ""
echo "Generate a defaults.yml file first updating some parameters with a yaml string..."
echo "/tmp/command_line/defaults.yml (a has been updated)"
echo "----------------------------------------------------"
echo "$(cat /tmp/command_line/defaults.yml)"

Updating parameters {'a': 'updated'}

Generate a defaults.yml file first updating some parameters with a yaml string...
/tmp/command_line/defaults.yml (a has been updated)
----------------------------------------------------
created: 03:42PM September 18, 2019  #  The date the last time the parameters of the model were updated.
version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/projects/rad2sim2rad/docs/experiment.py
  class: AnExperiment
  git:
    local: /Users/robweston/projects/rad2sim2rad
    remote: ssh://git@mrgbuild.robots.ox.ac.uk:7999/~robw/rad2sim2rad.git
    commit: 70775458a7599e70540e7f21a408435950320fac
    branch: master
a: updated #  A parameter (default='h')
b: 17 #  Another parameter (default=17)


In [19]:
%%bash
# Registering an experiment
rm -f /tmp/command_line/an_experiment/params.yml

python experiment.py --register /tmp/command_line an_experiment
echo ""
echo "/tmp/command_line/an_experiment/params.yml"
echo "----------------------------------------------------"
echo "$(cat /tmp/command_line/an_experiment/params.yml)"


/tmp/command_line/an_experiment/params.yml
----------------------------------------------------
root: /tmp/command_line  #  The root directory to which the experiment belongs (should not be set)
name: an_experiment #  The name of the experiment (should not be set)
status: registered #  The status of the experiment (one of ['default' | 'created' | 'running' | 'error' | 'finished'])
created: 07:47PM September 18, 2019 #  The date the last time the parameters of the model were updated.
purpose: '' #  The purpose for the experiment (should not be set)
messages: {} #  A dictionary of messages which are able to vary throughout the experiment (should not be set)
version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/experiment.py
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
    branch: mast

In [20]:
%%bash
# Execute an experiment
python experiment.py --execute /tmp/command_line/an_experiment/params.yml

echo ""
echo "After executing the experiment..."
echo "/tmp/command_line/an_experiment/params.yml"
echo "----------------------------------------------------"
echo "$(cat /tmp/command_line/an_experiment/params.yml)"

The experiment state inside run is running for experiment an_experiment, a = h, b = 17

After executing the experiment...
/tmp/command_line/an_experiment/params.yml
----------------------------------------------------
root: /tmp/command_line  #  The root directory to which the experiment belongs (should not be set)
name: an_experiment #  The name of the experiment (should not be set)
status: finished #  The status of the experiment (one of ['default' | 'created' | 'running' | 'error' | 'finished'])
created: 07:47PM September 18, 2019 #  The date the last time the parameters of the model were updated.
purpose: '' #  The purpose for the experiment (should not be set)
messages: #  A dictionary of messages which are able to vary throughout the experiment (should not be set)
  Hi: I am running!
version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/experiment.py
  class: AnExperiment
  git:
    local: /

## The Experiment Manager
The experiment manager is a python command line tool for managing a set of experiments which all share execution code (a `script.sh`) which takes as input parameters (`defaults.yml`). By changing the parameters, each experiment is uniquely defined. Firstly, lets create a root experiment repository:

In [21]:
!python experiment.py --to_root /tmp/experiment_manager/root

#!/bin/bash
# File generated on the 07:47PM September 18, 2019
# GIT:
# - repo /Users/robweston/xmen
# - branch master
# - remote https://github.com/robw4/xmen.git
# - commit 7d9d2f875219b7d14bf6389fb6527d2425adc66a

export PYTHONPATH="${PYTHONPATH}:/Users/robweston/xmen/examples"
python /Users/robweston/xmen/examples/experiment.py --execute ${1}


The above gives a bash script automatically generated in order to run an experiment. This is saved as `script.sh` in the root experiment repository along with a `default.yml` file.

In [22]:
!ls -l /tmp/experiment_manager/root

total 24
-rw-r--r--  1 robweston  wheel  517 18 Sep 19:47 defaults.yml
-rw-r--r--  1 robweston  wheel  330 18 Sep 19:15 experiment.yml
-rw-r--r--  1 robweston  wheel  348 18 Sep 19:47 script.sh


For ease of use we first create an alias to the experiment manager. You are encouraged to do this in your `~/.bashrc` or equivalent. Unfortunately ipython does not register previously defined aliases so we need to redefine it here. We also move to the root repository:

In [23]:
alias xmen ~/xmen/python/xmen/experiment_manager.py

First we navigate to the root directory:

In [24]:
%cd /tmp/experiment_manager/root
!rm -f experiment.yml

/private/tmp/experiment_manager/root


Next we initialise the experiment manager

In [25]:
%xmen init

Initilialising the experiment parameter in the root directory produces a `experiment.yml` file. The `exp list` command provides a helpfull output fo visualisnd the state of the current experiements:

In [26]:
%xmen list

No experiments currently registered!

Defaults: 
- created 07:47PM September 18, 2019
- module /Users/robweston/xmen/examples/experiment.py
- class AnExperiment
- git:
   local: /Users/robweston/xmen
   commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
   remote: https://github.com/robw4/xmen.git


Multiple experiments are registered with the experiment manager in a single call through the `register` command:

In [27]:
xmen register '{a: first | second, b: 4 | 5}' 'intial_test'

The special charcter `|` is interpreted as an or operator. Experiments are registered for every possible combination of parameters appearing with `|`. For example in the above 4 experiments have been registered:

In [28]:
%xmen list

       overrides      purpose                     created      status messages                                    commit
0   a:first__b:4  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
1   a:first__b:5  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
2  a:second__b:4  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
3  a:second__b:5  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a

Defaults: 
- created 07:47PM September 18, 2019
- module /Users/robweston/xmen/examples/experiment.py
- class AnExperiment
- git:
   local: /Users/robweston/xmen
   commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
   remote: https://github.com/robw4/xmen.git


You are also allowed to register multiple experiments with the same overrides. In this case there name is appended by an index `_1` `_2` etc:

In [29]:
%xmen register '{a: first, b: 4}' 'intial_test'
%xmen register '{a: first, b: 4}' 'intial_test'
%xmen list

        overrides      purpose                     created      status messages                                    commit
0    a:first__b:4  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
1    a:first__b:5  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
2   a:second__b:4  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
3   a:second__b:5  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
4  a:first__b:4_1  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
5  a:first__b:4_2  intial_test  07:47PM September 18, 2019  registered       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a

Defaults: 
- created 07:47PM September 18, 2019
- module /Users/robweston/xmen/examples/experiment.py
- class AnExperiment
- git:
   local: /User

Experiments can be run either individually or all together with several prefixes:

In [30]:
%xmen run 'a:second__b:4' sh


Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:second__b:4/params.yml
The experiment state inside run is running for experiment a:second__b:4, a = second, b = 4


In [31]:
%xmen list

        overrides      purpose                     created      status                 messages                                    commit
0    a:first__b:4  intial_test  07:47PM September 18, 2019  registered                       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
1    a:first__b:5  intial_test  07:47PM September 18, 2019  registered                       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
2   a:second__b:4  intial_test  07:47PM September 18, 2019    finished  {'Hi': 'I am running!'}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
3   a:second__b:5  intial_test  07:47PM September 18, 2019  registered                       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
4  a:first__b:4_1  intial_test  07:47PM September 18, 2019  registered                       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a
5  a:first__b:4_2  intial_test  07:47PM September 18, 2019  registered                       {}  7d9d2f875219b7d14bf6389fb6527d2425adc66a

Defaults: 
- created 07:47PM Sept

In [32]:
%xmen run '*' sh
%xmen list


Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:first__b:4/params.yml
The experiment state inside run is running for experiment a:first__b:4, a = first, b = 4

Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:first__b:4_1/params.yml
The experiment state inside run is running for experiment a:first__b:4_1, a = first, b = 4

Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:first__b:4_2/params.yml
The experiment state inside run is running for experiment a:first__b:4_2, a = first, b = 4

Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:first__b:5/params.yml
The experiment state inside run is running for experiment a:first__b:5, a = first, b = 5

Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:second__b:5/params.yml
The experiment state inside ru

Lets now clean up the experiment repository:

In [33]:
# First we unlink all the experiments
%xmen unlink '*'

Removing Experiments...
/private/tmp/experiment_manager/root/a:first__b:4
/private/tmp/experiment_manager/root/a:first__b:4_1
/private/tmp/experiment_manager/root/a:first__b:4_2
/private/tmp/experiment_manager/root/a:first__b:5
/private/tmp/experiment_manager/root/a:second__b:4
/private/tmp/experiment_manager/root/a:second__b:5
Note models are removed from the experiment list only. To remove the model directories runexperiment rm


In [34]:
#Now we remove the experiment folders 
#WARNING: THIS WILL ALL EXPERIMENTS THAT ARE NO LONGER LINKED TO THE EXPERIMENT MANAGER
%xmen rm '*'
!echo ""
%xmen list

/private/tmp/experiment_manager/root/a:first__b:4
/private/tmp/experiment_manager/root/a:first__b:4_1
/private/tmp/experiment_manager/root/a:first__b:4_2
/private/tmp/experiment_manager/root/a:first__b:5
/private/tmp/experiment_manager/root/a:second__b:4
/private/tmp/experiment_manager/root/a:second__b:5

No experiments currently registered!

Defaults: 
- created 07:47PM September 18, 2019
- module /Users/robweston/xmen/examples/experiment.py
- class AnExperiment
- git:
   local: /Users/robweston/xmen
   commit: 7d9d2f875219b7d14bf6389fb6527d2425adc66a
   remote: https://github.com/robw4/xmen.git


## The TypedMeta metaclass
The `TypedMeta` meta class stremalines the definition of objects whith many attirbutes that most of the time will simply just take there default value. For example consider the following class definition:

In [35]:
class A(object):
    def __init__(self, a=3, b=4, c=3, d=4, e=5, f=6, g=7):
        """An object with some attributes:
        
        Parameters:
            a (int): The first attribute
            b (int): The second attribute
            c (int): The third attribute 
            d (int): The fourth attribute
            e (int): The fith attribute
            g (int): The sixth attribute
        """
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        self.f = f
        self.g = g

For a large number of parameters defining parameters in the args, docstrings and attributes quickly becomes tiresome. We could instead think about inferring attributes directly from the arguments of a function. Maybe by updating `self.__dict__` directly after using `argparse` to get the parameters from the function definition or simply only passing `**kwargs` to `__init__`. This seems quite convoluted and will break code completion for many interpreters.

This is where the `TypedMeta` metaclass comes in. Instead we define objects like:

In [36]:
from xmen.utils import TypedMeta
class AWithMeta(object, metaclass=TypedMeta):
    def __init__(self):
        """An object with some attributes"""
        self.a: int = 1   # The first attribute
        self.b: int = 2   # The second attribute
        self.c: int = 3   # The third attribute 
        self.d: int = 4   # The fourth attribute
        self.e: int = 5   # The fith attribute
        self.f: int = 6   # The sixth attribute
        self.g: int = 7   # The seventh attribute

Here we have made use of typed attributes (introduced in python >= 3). In order to allow the defualt values to be overidden we could also considered passing an option keywords arg `**kwargs` and updating the attribute dictionary directly (updating only on keys correpsonding to valid attributes). The `TypedMeta` metaclass automatically adds attributes to the docstring of `AWithMeta` and `AWithMeta.__init__`:

In [37]:
# The doc string of an experiment object is updated
print(AWithMeta.__init__.__doc__)

An object with some attributes

Parameters:
    a (int): The first attribute (default=1)
    b (int): The second attribute (default=2)
    c (int): The third attribute (default=3)
    d (int): The fourth attribute (default=4)
    e (int): The fith attribute (default=5)
    f (int): The sixth attribute (default=6)
    g (int): The seventh attribute (default=7)


The typed Meta class works with both typed and non-typed attribute definitions with or without comments. It will do the best it can with what it has. For example:

In [38]:
class BWithMeta(object, metaclass=TypedMeta):
    def __init__(self):
        """An object with some attributes"""
        self.a: float = 1   # The first attribute
        self.b: int = 2
        self.c = 3   # The third attribute 
        self.d = 4

print(BWithMeta.__doc__)



Parameters:
    a (float): The first attribute (default=1)
    b (int): (default= 2)
    c : The third attribute (default=3)
    d : (default= 4)


> The TypedMeta class is currently only compatible with comments defined on a single line. You are recommended to specify longer comments in the doc-string directly